## Associating points to the grades in order to calculate the Loan Score

In [0]:
spark.conf.set("spark.sql.unacceptable_rated_pts", 0)
spark.conf.set("spark.sql.very_bad_rated_pts", 100)
spark.conf.set("spark.sql.bad_rated_pts", 250)
spark.conf.set("spark.sql.good_rated_pts", 500)
spark.conf.set("spark.sql.very_good_rated_pts", 650)
spark.conf.set("spark.sql.excellent_rated_pts", 800)

In [0]:
spark.conf.set("spark.sql.unacceptable_grade_pts", 750)
spark.conf.set("spark.sql.very_bad_grade_pts", 1000)
spark.conf.set("spark.sql.bad_grade_pts", 1500)
spark.conf.set("spark.sql.good_grade_pts", 2000)
spark.conf.set("spark.sql.very_good_grade_pts", 2500)

## The tables required to calculate the Loan Score

customers_new 

loans

loans_repayments

loans_defaulters_delinq_new

loans_defaulters_detail_red_enq_new

## Loan Score Calculation Criteria 1: Payment History(ph)

In [0]:
bad_customer_data_final_df = spark.read \
.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.load("/mnt/Lendingclub/Lendingclub/Lendingclub/bad/bad_customer_data_final")

In [0]:
bad_customer_data_final_df.createOrReplaceTempView("bad_data_customer")

In [0]:
ph_df = spark.sql("select c.member_id, \
   case \
   when p.last_payment_amount < (c.monthly_installment * 0.5) then ${spark.sql.very_bad_rated_pts} \
   when p.last_payment_amount >= (c.monthly_installment * 0.5) and p.last_payment_amount < c.monthly_installment then ${spark.sql.very_bad_rated_pts} \
   when (p.last_payment_amount = (c.monthly_installment)) then ${spark.sql.good_rated_pts} \
   when p.last_payment_amount > (c.monthly_installment) and p.last_payment_amount <= (c.monthly_installment * 1.50) then ${spark.sql.very_good_rated_pts} \
   when p.last_payment_amount > (c.monthly_installment * 1.50) then ${spark.sql.excellent_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   end as last_payment_pts, \
   case \
   when p.total_payment_received >= (c.funded_amount * 0.50) then ${spark.sql.very_good_rated_pts} \
   when p.total_payment_received < (c.funded_amount * 0.50) and p.total_payment_received > 0 then ${spark.sql.good_rated_pts} \
   when p.total_payment_received = 0 or (p.total_payment_received) is null then ${spark.sql.unacceptable_rated_pts} \
   end as total_payment_pts \
from lending_club.loans_repayments p \
inner join lending_club.loans c on c.loan_id = p.loan_id where member_id NOT IN (select member_id from bad_data_customer)")

In [0]:
ph_df.createOrReplaceTempView("ph_pts")

In [0]:
display(spark.sql("select * from ph_pts").limit(20))

member_id,last_payment_pts,total_payment_pts
00720193212c0cb6fa8fe92b0bcffd5ce7b9424f8fc671242e0b9975c0dbe676,800,650
b1865c807a867bc0031965129690d60eef86f5702b8acb2c6ac25dbee1e17386,800,650
8b161e0f538f5ba0280f822611d6c5cd93b24a51c99bc1f8db61dc9a712f6532,500,650
480514c1d6b3cf95abb6cbd85947ab64e0e18df431005cb556c3a4217e777c40,500,650
3dae0d924bcb74693dd021137677613e4924c75bb645cffc6c37c94e2498681e,500,650
15d328ff0703d79de7b73fd4d8ae695d3a4c7ef0e0af2afe1b8f7477c3a9bc53,500,650
18e6d99ee96fca9043e6b0826454e8f31210786578db2ac309b4e828aeb4604d,500,650
2e89013db4b84924ddd9f7d3ae1df596b088a6e5972777e5820c1e7b10f36aa6,800,650
26add02fec67582541c6e8a3bd5b364253d3b170c8026abf24e372763a30aa89,500,650
6d34b3bfafc54a21fcd76691e125569c8b8398b343c20130bac10867fe56d3d1,500,650


##  Loan Score Calculation Criteria 2: Loan Defaulters History(ldh)

In [0]:
ldh_ph_df = spark.sql(
    "select p.*, \
    CASE \
    WHEN d.delinq_2yrs = 0 THEN ${spark.sql.excellent_rated_pts} \
    WHEN d.delinq_2yrs BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
    WHEN d.delinq_2yrs BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
    WHEN d.delinq_2yrs > 5 OR d.delinq_2yrs IS NULL THEN ${spark.sql.unacceptable_grade_pts} \
    END AS delinq_pts, \
    CASE \
    WHEN l.pub_rec = 0 THEN ${spark.sql.excellent_rated_pts} \
    WHEN l.pub_rec BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
    WHEN l.pub_rec BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
    WHEN l.pub_rec > 5 OR l.pub_rec IS NULL THEN ${spark.sql.very_bad_rated_pts} \
    END AS public_records_pts, \
    CASE \
    WHEN l.pub_rec_bankruptcies = 0 THEN ${spark.sql.excellent_rated_pts} \
    WHEN l.pub_rec_bankruptcies BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
    WHEN l.pub_rec_bankruptcies BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
    WHEN l.pub_rec_bankruptcies > 5 OR l.pub_rec_bankruptcies IS NULL THEN ${spark.sql.very_bad_rated_pts} \
    END as public_bankruptcies_pts, \
    CASE \
    WHEN l.inq_last_6mths = 0 THEN ${spark.sql.excellent_rated_pts} \
    WHEN l.inq_last_6mths BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
    WHEN l.inq_last_6mths BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
    WHEN l.inq_last_6mths > 5 OR l.inq_last_6mths IS NULL THEN ${spark.sql.unacceptable_rated_pts} \
    END AS enq_pts \
    FROM lending_club.loans_defaulters_detail_rec_enq_new l \
    INNER JOIN lending_club.loans_defaulters_delinq_new d ON d.member_id = l.member_id  \
    INNER JOIN ph_pts p ON p.member_id = l.member_id where l.member_id NOT IN (select member_id from bad_data_customer)")

In [0]:
ldh_ph_df.createOrReplaceTempView("ldh_ph_pts")

In [0]:
display(spark.sql("select * from ldh_ph_pts").limit(20))

member_id,last_payment_pts,total_payment_pts,delinq_pts,public_records_pts,public_bankruptcies_pts,enq_pts
0000036e9afe019a699318476a8a0369a6077eaec49941fe3d2d20a8337a29da,800,650,250,800,800,100
000152208b3e77b5b312ec37200c9e42bf55e919832deb42316f81b07b451071,500,650,800,800,800,800
000170b4ccb292792210dbabd9cf19972e101df6fc5afba151fa89a047c4ae61,800,650,250,800,800,250
0001cfa200f7480b9da8567b58c35f83eaa9b9582926103959e66f3479c83302,500,650,250,250,250,250
00024adf1230710bd0ef07843ff9e1e47e8427e9054d6eea02c3ae61854f92ea,500,500,250,800,800,800
00026136ec721b938d8742c6eb93160e0f54b8e02c75ccdbed010e3c619f9e82,500,500,250,800,800,250
0002b05a715d725840d6fc999c93896f7336dff953f621c8132f2788282a3999,500,500,800,800,800,800
00030e831c078f92ae63c5d683defc180eedf3c72726e244b017fefe3d10e256,500,650,800,800,800,800
00039d9a16af9223f4711c2fd60f8fb4436a4f03b4490cfbac567885fa8dae21,500,500,800,800,800,800
00040e9425ee269ebfc64981b80d9867f9d11adc07ca0a8ce9c1b0d90e64a8e2,500,650,250,250,250,800


## Loan Score Calculation Criteria : Financial Health(fh)

In [0]:
fh_ldh_ph_df = spark.sql("select ldef.*, \
   CASE \
   WHEN LOWER(l.loan_status) LIKE '%fully paid%' THEN ${spark.sql.excellent_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%current%' THEN ${spark.sql.good_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%in grace period%' THEN ${spark.sql.bad_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%late (16-30 days)%' OR LOWER(l.loan_status) LIKE '%late (31-120 days)%' THEN ${spark.sql.very_bad_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%charged off%' THEN ${spark.sql.unacceptable_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   END AS loan_status_pts, \
   CASE \
   WHEN LOWER(a.home_ownership) LIKE '%own' THEN ${spark.sql.excellent_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%rent' THEN ${spark.sql.good_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%mortgage' THEN ${spark.sql.bad_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%any' OR LOWER(a.home_ownership) IS NULL THEN ${spark.sql.very_bad_rated_pts} \
   END AS home_pts, \
   CASE \
   WHEN l.funded_amount <= (a.tot_hi_cred_lim * 0.10) THEN ${spark.sql.excellent_rated_pts} \
   WHEN l.funded_amount > (a.tot_hi_cred_lim * 0.10) AND l.funded_amount <= (a.tot_hi_cred_lim * 0.20) THEN ${spark.sql.very_good_rated_pts} \
   WHEN l.funded_amount > (a.tot_hi_cred_lim * 0.20) AND l.funded_amount <= (a.tot_hi_cred_lim * 0.30) THEN ${spark.sql.good_rated_pts} \
   WHEN l.funded_amount > (a.tot_hi_cred_lim * 0.30) AND l.funded_amount <= (a.tot_hi_cred_lim * 0.50) THEN ${spark.sql.bad_rated_pts} \
   WHEN l.funded_amount > (a.tot_hi_cred_lim * 0.50) AND l.funded_amount <= (a.tot_hi_cred_lim * 0.70) THEN ${spark.sql.very_bad_rated_pts} \
   WHEN l.funded_amount > (a.tot_hi_cred_lim * 0.70) THEN ${spark.sql.unacceptable_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   END AS credit_limit_pts, \
   CASE \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A1' THEN ${spark.sql.excellent_rated_pts} \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A2' THEN (${spark.sql.excellent_rated_pts} * 0.95) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A3' THEN (${spark.sql.excellent_rated_pts} * 0.90) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A4' THEN (${spark.sql.excellent_rated_pts} * 0.85) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A5' THEN (${spark.sql.excellent_rated_pts} * 0.80) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B1' THEN (${spark.sql.very_good_rated_pts}) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B2' THEN (${spark.sql.very_good_rated_pts} * 0.95) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B3' THEN (${spark.sql.very_good_rated_pts} * 0.90) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B4' THEN (${spark.sql.very_good_rated_pts} * 0.85) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B5' THEN (${spark.sql.very_good_rated_pts} * 0.80) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C1' THEN (${spark.sql.good_rated_pts}) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C2' THEN (${spark.sql.good_rated_pts} * 0.95) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C3' THEN (${spark.sql.good_rated_pts} * 0.90) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C4' THEN (${spark.sql.good_rated_pts} * 0.85) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C5' THEN (${spark.sql.good_rated_pts} * 0.80) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D1' THEN (${spark.sql.bad_rated_pts}) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D2' THEN (${spark.sql.bad_rated_pts} * 0.95) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D3' THEN (${spark.sql.bad_rated_pts} * 0.90) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D4' THEN (${spark.sql.bad_rated_pts} * 0.85) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D5' THEN (${spark.sql.bad_rated_pts} * 0.80) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E1' THEN (${spark.sql.very_bad_rated_pts}) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E2' THEN (${spark.sql.very_bad_rated_pts} * 0.95) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E3' THEN (${spark.sql.very_bad_rated_pts} * 0.90) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E4' THEN (${spark.sql.very_bad_rated_pts} * 0.85) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E5' THEN (${spark.sql.very_bad_rated_pts} * 0.80) \
   WHEN (a.grade) in ('F', 'G') THEN (${spark.sql.unacceptable_rated_pts}) \
   END AS grade_pts \
   FROM ldh_ph_pts ldef \
   INNER JOIN lending_club.loans l ON ldef.member_id = l.member_id \
   INNER JOIN lending_club.customers_new a ON a.member_id = ldef.member_id where ldef.member_id NOT IN (select member_id from bad_data_customer)") 

In [0]:
fh_ldh_ph_df.createOrReplaceTempView("fh_ldh_ph_pts")

#### Final loan score calculation by considering all the 3 criterias with the following %**

#### 1. Payment History = 20%
#### 2. Loan Defaults = 45%
#### 3. Financial Health = 35%

In [0]:
loan_score = spark.sql("SELECT member_id, \
((last_payment_pts+total_payment_pts)*0.20) as payment_history_pts, \
((delinq_pts + public_records_pts + public_bankruptcies_pts + enq_pts) * 0.45) as defaulters_history_pts, \
((loan_status_pts + home_pts + credit_limit_pts + grade_pts)*0.35) as financial_health_pts \
FROM fh_ldh_ph_pts")

In [0]:
display(loan_score.limit(20))

member_id,payment_history_pts,defaulters_history_pts,financial_health_pts
0000036e9afe019a699318476a8a0369a6077eaec49941fe3d2d20a8337a29da,290.00,877.50,822.5000
000152208b3e77b5b312ec37200c9e42bf55e919832deb42316f81b07b451071,230.00,1440.00,665.0000
000170b4ccb292792210dbabd9cf19972e101df6fc5afba151fa89a047c4ae61,290.00,945.00,852.2500
0001cfa200f7480b9da8567b58c35f83eaa9b9582926103959e66f3479c83302,230.00,450.00,665.0000
00024adf1230710bd0ef07843ff9e1e47e8427e9054d6eea02c3ae61854f92ea,200.00,1192.50,682.5000
00026136ec721b938d8742c6eb93160e0f54b8e02c75ccdbed010e3c619f9e82,200.00,945.00,653.6250
00030e831c078f92ae63c5d683defc180eedf3c72726e244b017fefe3d10e256,230.00,1440.00,595.8750
0004656412a18b9c138fac6b7efe457401305d08062596240c1298bb1a8a37b7,290.00,1192.50,988.7500
0004f290201c29d93a8b365965e63447a3fa74feb61640fcf54b390ad822fa6f,200.00,1440.00,682.5000
00055f1f392f63672ee4c74393b994fa5403b9e1949b34a25c8d083cb6c30161,290.00,1440.00,875.8750


In [0]:
final_loan_score = loan_score.withColumn('loan_score', loan_score.payment_history_pts + loan_score.defaulters_history_pts + loan_score.financial_health_pts)

In [0]:
final_loan_score.createOrReplaceTempView("loan_score_eval")

In [0]:
loan_score_final = spark.sql("select ls.*, \
case \
WHEN loan_score > ${spark.sql.very_good_grade_pts} THEN 'A' \
WHEN loan_score <= ${spark.sql.very_good_grade_pts} AND loan_score > ${spark.sql.good_grade_pts} THEN 'B' \
WHEN loan_score <= ${spark.sql.good_grade_pts} AND loan_score > ${spark.sql.bad_grade_pts} THEN 'C' \
WHEN loan_score <= ${spark.sql.bad_grade_pts} AND loan_score  > ${spark.sql.very_bad_grade_pts} THEN 'D' \
WHEN loan_score <= ${spark.sql.very_bad_grade_pts} AND loan_score > ${spark.sql.unacceptable_grade_pts} THEN 'E'  \
WHEN loan_score <= ${spark.sql.unacceptable_grade_pts} THEN 'F' \
end as loan_final_grade \
from loan_score_eval ls")

In [0]:
loan_score_final.createOrReplaceTempView("loan_final_table")

In [0]:
display(spark.sql("select * from loan_final_table").limit(20))

member_id,payment_history_pts,defaulters_history_pts,financial_health_pts,loan_score,loan_final_grade
0000036e9afe019a699318476a8a0369a6077eaec49941fe3d2d20a8337a29da,290.00,877.50,822.5000,1990.0000,C
000152208b3e77b5b312ec37200c9e42bf55e919832deb42316f81b07b451071,230.00,1440.00,665.0000,2335.0000,B
000170b4ccb292792210dbabd9cf19972e101df6fc5afba151fa89a047c4ae61,290.00,945.00,852.2500,2087.2500,B
0001cfa200f7480b9da8567b58c35f83eaa9b9582926103959e66f3479c83302,230.00,450.00,665.0000,1345.0000,D
00024adf1230710bd0ef07843ff9e1e47e8427e9054d6eea02c3ae61854f92ea,200.00,1192.50,682.5000,2075.0000,B
00026136ec721b938d8742c6eb93160e0f54b8e02c75ccdbed010e3c619f9e82,200.00,945.00,653.6250,1798.6250,C
00030e831c078f92ae63c5d683defc180eedf3c72726e244b017fefe3d10e256,230.00,1440.00,595.8750,2265.8750,B
0004656412a18b9c138fac6b7efe457401305d08062596240c1298bb1a8a37b7,290.00,1192.50,988.7500,2471.2500,B
0004f290201c29d93a8b365965e63447a3fa74feb61640fcf54b390ad822fa6f,200.00,1440.00,682.5000,2322.5000,B
00055f1f392f63672ee4c74393b994fa5403b9e1949b34a25c8d083cb6c30161,290.00,1440.00,875.8750,2605.8750,A


In [0]:
display(spark.sql("select count(*) from loan_final_table"))

count(1)
1102587


In [0]:
loan_score_final.write \
.format("csv") \
.mode("overwrite") \
.option("path", "/mnt/Lendingclub/Lendingclub/Lendingclub/processed/loan_score") \
.save()